# Visualizations

## Setup

In [ ]:
import matplotlib.pyplot as plt
from herbie import FastHerbie, paint, wgrib2, Herbie
from herbie.toolbox import EasyMap, ccrs, pc
import xarray as xr
from datetime import datetime
import sys
import os.path as osp
import pandas as pd
import numpy as np
sys.path.append("../src")
from utils import Dict, read_yml, str2time, print_dict_summary, read_pkl
from viz import map_var, create_gif

In [ ]:
data = pd.read_pickle("../data/ml_data_rocky24.pkl")
outdir = "../outputs/forecast_outputs"

In [ ]:
# Read HRRR Data
H = Herbie("2025-01-01", product="prs")
ds = H.xarray("(?:HGT|LAND):surface")
hrrr_ds = xr.open_dataset("../data/fm_preds_hrrr.nc")
hrrr_ds.herbie.crs = ds.herbie.crs

## Viz

### Animated Maps

In [ ]:
hrrr_ds["Ew"] = hrrr_ds.Ew.where(hrrr_ds.lsm > 0)
hrrr_ds["fm_preds"] = hrrr_ds.fm_preds.where(hrrr_ds.lsm > 0)

In [ ]:
mt_time = pd.to_datetime(hrrr_ds.valid_time.values).tz_localize("UTC").tz_convert("America/Denver")

In [ ]:
tstep = 0

t = hrrr_ds.valid_time[tstep]
t = f"{t.dt.year.item():04d}-{t.dt.month.item():02d}-{t.dt.day.item():02d} {t.dt.hour.item():02d}:{t.dt.minute.item():02d}:{t.dt.second.item():02d}"

map_var(hrrr_ds, "Ew", 
        time_step = tstep,
        legend_title = "Equilibrium Moisture Content (%)",
        title = f"Wetting Equilibrium HRRR Forecast at {t} UTC"
       )

In [ ]:
# Define time steps (e.g., first 10)
time_steps = range(0, 48)  
create_gif(hrrr_ds, "Ew", time_steps, 
           legend_title="Equilibrium Moisture Content (%)", gif_path='../outputs/eq.gif', duration=0.5)


In [ ]:
tstep = 0
time_steps = range(0, 48)  
t = hrrr_ds.valid_time[tstep]
t = f"{t.dt.year.item():04d}-{t.dt.month.item():02d}-{t.dt.day.item():02d} {t.dt.hour.item():02d}:{t.dt.minute.item():02d}:{t.dt.second.item():02d}"

create_gif(hrrr_ds, "fm_preds", time_steps, legend_title="Fuel Moisture Content (%)", gif_path='../outputs/fmc2.gif', duration=0.5)

In [ ]:
create_gif(hrrr_ds, "rh", time_steps, legend_title="Relative Humidity (%)", gif_path='../outputs/rh2.gif', duration=0.5)

In [ ]:
create_gif(hrrr_ds, "rain", time_steps, legend_title="Rain (mm/hr)", gif_path='../outputs/rain.gif', duration=0.5)

In [ ]:
create_gif(hrrr_ds, "temp", time_steps, legend_title="Air Temperature (K)", gif_path='../outputs/temp2.gif', duration=0.5)

In [ ]:
tstep = 0
time_steps = range(0, 48)  
t = hrrr_ds.valid_time[tstep]
t = f"{t.dt.year.item():04d}-{t.dt.month.item():02d}-{t.dt.day.item():02d} {t.dt.hour.item():02d}:{t.dt.minute.item():02d}:{t.dt.second.item():02d}"

create_gif(hrrr_ds, "wind", time_steps, legend_title="Wind Speed 10m (m/s)", gif_path='../outputs/wind2.gif', duration=0.5)

## Predictor Maps

Show that rain has discontinuities, rh and temp smooth.

In [ ]:
bbox = [37,-111,46,-95]

In [ ]:
tstep = 47
t = hrrr_ds.valid_time[tstep]
t = f"{t.dt.year.item():04d}-{t.dt.month.item():02d}-{t.dt.day.item():02d} {t.dt.hour.item():02d}:{t.dt.minute.item():02d}:{t.dt.second.item():02d}"

map_var(hrrr_ds, "rain", 
        time_step = tstep,
        legend_title = "Rain (mm/hr)",
        title = f"Rain HRRR Forecast at {t} UTC",
        save_path = "../outputs/rain_map.png"
       )

In [ ]:
map_var(hrrr_ds, "temp", 
        time_step = tstep,
        legend_title = "Temperature (k)",
        title = f"Air Temp HRRR Forecast at {t} UTC",
        save_path = "../outputs/temp_map.png"
       )

In [ ]:
map_var(hrrr_ds, "rh", 
        time_step = tstep,
        legend_title = "Relative Humidity (%)",
        title = f"Relative Humidity HRRR Forecast at {t} UTC",
        save_path = "../outputs/rh_map.png"
       )

In [ ]:
map_var(hrrr_ds, "Ew", 
        time_step = 0,
        legend_title = "Equilibrium Moisture Content (%)",
        title = f"Eq from HRRR Forecast at {t} UTC",
        save_path = "../outputs/eq_map.png"
       )

In [ ]:
hrrr_ds['longitude'] = xr.where(hrrr_ds.longitude > 180, hrrr_ds.longitude - 360, hrrr_ds.longitude)
lat = hrrr_ds.latitude
lon = hrrr_ds.longitude

mask = (
    (lat >= bbox[0]) & (lat <= bbox[2]) &
    (lon >= bbox[1]) & (lon <= bbox[3])
)

subset = hrrr_ds.where(mask)                    # apply mask
subset = subset.dropna(dim='x', how='all')      # drop fully-masked columns
subset = subset.dropna(dim='y', how='all')      # drop fully-masked rows
subset.herbie.crs = ds.herbie.crs

In [ ]:
# tstep=47
# t = hrrr_ds.valid_time[tstep]
# t = f"{t.dt.year.item():04d}-{t.dt.month.item():02d}-{t.dt.day.item():02d} {t.dt.hour.item():02d}:{t.dt.minute.item():02d}:{t.dt.second.item():02d}"

# map_var(subset, "rain", 
#         time_step = tstep,
#         legend_title = "Rain (mm/hr)",
#         title = f"Rain HRRR Forecast at {t} UTC - Rocky Mountain GACC",
#         save_path = "../outputs/rain_map_zoom.png"
#        )

In [ ]:
# map_var(subset, "temp", 
#         time_step = tstep,
#         legend_title = "Temperature (k)",
#         title = f"Temperature HRRR Forecast at {t} UTC - Rocky Mountain GACC|",
#         save_path = "../outputs/temp_map_zoom.png"
#        )

In [ ]:
# map_var(subset, "rh", 
#         time_step = tstep,
#         legend_title = "Relative Humidity (%)",
#         title = f"Relative Humidity HRRR Forecast at {t} UTC - Rocky Mountain GACC",
#         save_path = "../outputs/rh_map_zoom.png"
#        )

In [ ]:
map_var(subset, "Ew", 
        time_step = 0,
        legend_title = "Equilibrium Moisture Content (%)",
        title = f"Eq from HRRR Forecast at {t} UTC - Rocky Mountain GACC",
        save_path = "../outputs/eq_map_zoom.png"
       )

### Timeseries Plots

### Plot fm

In [ ]:
start = str2time("2024-01-01T00:00:00Z")
end = str2time("2024-01-02T23:00:00Z")
st = "SOPC2"

In [ ]:
df= data[st]["data"]
df = df[(df.date_time >= start) & (df.date_time <= end)]

In [ ]:
plt.rcParams.update({'font.size': 16})  # or try 12, 16, etc.

fig, axs = plt.subplots(4, 1, figsize=(12, 8), sharex=True, constrained_layout=True)
axs[0].set_title(f"HRRR Weather Forecast at RAWS {st}")
axs[0].plot(df.date_time, df.Ed, color='#EF847C', label="Eq")
axs[0].set_ylabel("Eq Moisture (%)")

axs[1].plot(df.date_time, df.wind, color="gray", label="Wind")
axs[1].set_ylabel("Wind Speed (m/s)")

axs[2].plot(df.date_time, df.rain, color="blue", label="Rain")
axs[2].set_ylabel("Rain (mm/hr)")

axs[3].plot(df.date_time, df.elev, color="k", label="Elevation")
axs[3].set_ylabel("Elevation (m)")

import matplotlib.lines as mlines
ellipsis_line = mlines.Line2D([], [], color='none', label='...')


lines = []
labels = []
for ax in axs:
    for line in ax.get_lines():
        lines.append(line)
        labels.append(line.get_label())
# Dummy entry to make ellipses
lines.append(ellipsis_line)
labels.append(r'$\vdots$')

# fig.tight_layout()
fig.legend(lines, labels, loc='upper left', bbox_to_anchor=(1, 1))  # right of figure


# plt.show()
plt.savefig("../outputs/stacked_weather.png", bbox_inches='tight', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 2))  # same width, single subplot height

# Example plot
ax.plot(df.date_time, df.fm, color="#468a29")
ax.set_ylabel("FMC (%)")
ax.set_title(f"Observed FMC at RAWS {st}")
plt.xticks(rotation=90)
# plt.show()
plt.savefig("../outputs/fmc_ts_for_stacked.png", bbox_inches='tight', dpi=300)